# Инициализация

Инициализация весов в нейронной сети позволит улучшить и ускорить процесс обучения.
В этой лабораторной работе рассматриваются, как разные инициализации приводят к разным результатам.

**В рамках данной лабораторной работы будут приобретены следующие навыки (знания):**
- Реализация ускорения сходимости градиентного спуска
- Инициализация весов для увеличения шансов сходимости градиентного спуска к более низкой ошибке обучения

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на основной курс (для желающих получить дополнительный сертификаты): https://www.coursera.org/specializations/deep-learning

## 1 - Пакеты/Библиотеки

Первоначально необходимо запустить ячейку ниже, чтобы импортировать все пакеты, которые вам понадобятся во время лабораторной работы.
- [numpy](www.numpy.org) является основным пакетом для научных вычислений в Python.
- [matplotlib](http://matplotlib.org) это пакет для отрисовки графиков в Python.
- [sklearn](http://scikit-learn.org/stable/) предоставляет простые и эффективные инструменты для построения моделей и анализа данных.
- init_utils предоставляет различные полезные функции, используемые в данной лабораторной работе.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
from init_utils import sigmoid, relu, compute_loss, forward_propagation, backward_propagation
from init_utils import update_parameters, predict, load_dataset, plot_decision_boundary, predict_dec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 6.0) # размер графика
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Загрузка и отображение данных
train_X, train_Y, test_X, test_Y = load_dataset()

**Задача**: Построить классификатор, который разделяет красные и синие точки

## 1 - Модель нейронной сети

В лабораторной работе используется 3-слойнная нейронная сеть (уже реализована).
Вот методы инициализации, с которыми необходимо будет экспериментировать:
- *Инициализация в ноль* -- установить `initialization = "zeros"` как входной аргумент функции.
- *Случайная инициализация* -- установить `initialization = "random"` как входной аргумент функции. Это инициализирует веса к большим случайным значениям.
- *He инициализация* -- установить `initialization = "he"` как входной аргумент функции. Это инициализирует веса к случайным значениям по правилу предложенному авторами статьи He et al., 2015. 

**Инструкции**: Пожалуйста, прочитайте приведенный ниже код и запустите его. В следующей части необходимо будет реализовать три метода инициализации для данной модели `model()`.

In [ ]:
def model(X, Y, learning_rate = 0.01, num_iterations = 15000, print_cost = True, initialization = "he"):
    """
    3-х слойнная нейронная сеть с архитектурой: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- матрица признаков (2, number of examples)
    Y -- вектор меток (0 - красная точка; 1 - голубая точка), с размером (1, number of examples)
    learning_rate -- скорость градиентного спуска 
    num_iterations -- количество итераций при обучении
    print_cost -- True печать каждые 100 шагов
    initialization -- тип инициализации весов ("zeros","random" или "he")
    
    Returns:
    parameters -- обученные параметры модели
    """
        
    grads = {}
    costs = [] # отслеживание потерь
    m = X.shape[1] # количество примеров
    layers_dims = [X.shape[0], 10, 5, 1]
    
    # Инициализация параметров
    if initialization == "zeros":
        parameters = initialize_parameters_zeros(layers_dims)
    elif initialization == "random":
        parameters = initialize_parameters_random(layers_dims)
    elif initialization == "he":
        parameters = initialize_parameters_he(layers_dims)

    # Градиентный спуск
    for i in range(0, num_iterations):

        # Прямое распространение: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        a3, cache = forward_propagation(X, parameters)
        
        # Потери
        cost = compute_loss(a3, Y)

        # Обратное распространение
        grads = backward_propagation(X, Y, cache)
        
        # Обновление параметров
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Печать текущей итерации
        if print_cost and i % 1000 == 0:
            print("Потери после итерации {}: {}".format(i, cost))
            costs.append(cost)
            
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations')
    plt.title("Размер градиентного шага =" + str(learning_rate))
    plt.show()
    return parameters

## 2 - Инициализация в ноль

Существует два типа параметров, которые инициализируются в нейронной сети:
- матрица весов $(W^{[1]}, W^{[2]}, W^{[3]}, ..., W^{[L-1]}, W^{[L]})$
- векора смещений $(b^{[1]}, b^{[2]}, b^{[3]}, ..., b^{[L-1]}, b^{[L]})$

**Упражнение** Реализовать следующую функцию для инициализации параметров в ноль.
Позже можно будет увидеть, что это работает плохо, так как процесс обучения весов не может "нарушить симметрию".
Используйте np.zeros((..,..)) с корректными размерами.

In [ ]:
# ОЦЕНИВАЕМОЕ: initialize_parameters_zeros 

def initialize_parameters_zeros(layers_dims):
    """
    Arguments:
    layer_dims -- python массив (list), который содержит информацию о размере каждого слоя.
    
    Returns:
    parameters -- python словарь с параметрами "W1", "b1", ..., "WL", "bL":
                    W1 -- матрица весов с размером 1-го слоя (layers_dims[1], layers_dims[0])
                    b1 -- вектор смещений 1-го слоя (layers_dims[1], 1)
                    ...
                    WL -- матрица весов с размером L-го слоя (layers_dims[L], layers_dims[L-1])
                    bL -- вектор смещений L-го слоя (layers_dims[L], 1)
    """
    
    parameters = {}
    L = len(layers_dims) # количество слоёв в нейронной сети
    
    for l in range(1, L):
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 2 строки кода)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    return parameters

In [ ]:
parameters = initialize_parameters_zeros([3,2,1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Ожидаемый результат**:

<table> 
    <tr>
    <td>
    **W1**
    </td>
        <td>
    [[ 0.  0.  0.]
 [ 0.  0.  0.]]
    </td>
    </tr>
    <tr>
    <td>
    **b1**
    </td>
        <td>
    [[ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    **W2**
    </td>
        <td>
    [[ 0.  0.]]
    </td>
    </tr>
    <tr>
    <td>
    **b2**
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>

</table> 

Запустите следующий код для обучения модели на 15 000 итерациях с использованием инициализации в ноль.

In [ ]:
parameters = model(train_X, train_Y, initialization = "zeros")
print ("Обучающий датасет:")
predictions_train = predict(train_X, train_Y, parameters)
print ("Тестовый датасет:")
predictions_test = predict(test_X, test_Y, parameters)

Качество плохое, и размер потерь не уменьшается, и алгоритм работает не лучше, чем случайное угадывание. 

Почему? 

Необходимо рассмотреть детали совершаемых предсказаний и границы принятия решений:

In [ ]:
print ("predictions_train = " + str(predictions_train))
print ("predictions_test = " + str(predictions_test))

In [ ]:
plt.title("Модель с инициализацией в ноль")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y[0])

Модель предсказывает 0 для каждого примера. 

В общем случае инициализация всех весов до нуля приводит к тому, что сеть не может нарушить симметрию.
Это означает, что каждый нейрон в каждом слое будет изучать одно и то же, и вы можете с таким же успехом обучать нейронную сеть с $n^{[l]}=1$ для каждого слоя, и эта сеть не более мощна, чем линейный классификатор, такой как логистическая регрессия.

<font color='blue'>

**Что необходимо помнить:**
    
- Матрица весов $W^{[l]}$ должна быть инициализирована случайным образом для нарушения симметрии.
- Однако можно инициализировать смещения $b^{[l]}$ в ноль. Симметрия все ещё будет нарушена, пока $W^{[l]}$ инициализируется случайным образом.

## 3 - Случайная инициализация

Чтобы нарушить симметрию, давайте произвольно инициализируем веса. В этом упражнении продемонстрирован процесс того, что происходит, если веса инициализируются случайным образом, но с очень большими значениями.

**Упражнение** Реализуйте следующую функцию для инициализации весов до больших случайных значений (масштабированных на \*10) и смещений в ноль. Используйте `np.random.randn(..,..) * 10` для весов и `np.zeros((.., ..))` для смещений. Мы фиксируем случайную инициализацию `np.random.seed(..)`  чтобы убедиться, что ваши "случайные" веса совпадают с ответами, поэтому не переживайте, если при запуске несколько раз код дает всегда одни и те же начальные значения параметров.

In [ ]:
# ОЦЕНИВАЕМОЕ: initialize_parameters_random

def initialize_parameters_random(layers_dims):
    """
    Arguments:
    layer_dims -- python массив (list), который содержит информацию о размере каждого слоя.
    
    Returns:
    parameters -- python словарь с параметрами "W1", "b1", ..., "WL", "bL":
                    W1 -- матрица весов с размером 1-го слоя (layers_dims[1], layers_dims[0])
                    b1 -- вектор смещений 1-го слоя (layers_dims[1], 1)
                    ...
                    WL -- матрица весов с размером L-го слоя (layers_dims[L], layers_dims[L-1])
                    bL -- вектор смещений L-го слоя (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims)
    for l in range(1, L):
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 2 строки кода)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###

    return parameters

In [ ]:
parameters = initialize_parameters_random([3, 2, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Ожидаемый результат**:

<table> 
    <tr>
    <td>
    **W1**
    </td>
        <td>
    [[ 17.88628473   4.36509851   0.96497468]
 [-18.63492703  -2.77388203  -3.54758979]]
    </td>
    </tr>
    <tr>
    <td>
    **b1**
    </td>
        <td>
    [[ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    **W2**
    </td>
        <td>
    [[-0.82741481 -6.27000677]]
    </td>
    </tr>
    <tr>
    <td>
    **b2**
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>

</table> 

Запустите следующий код для обучения модели на 15 000 итерациях с использованием случайной инициализации.

In [ ]:
parameters = model(train_X, train_Y, initialization = "random")
print ("Обучающий датасет:")
predictions_train = predict(train_X, train_Y, parameters)
print ("Тестовый датасет:")
predictions_test = predict(test_X, test_Y, parameters)

Если вы видите "inf" как значение функции потерь после итерации 0, это происходит из-за численного округления; более сложная численная реализация исправит это. 

В любом случае можно наблюдать нарушение симметрии, и это дает лучшие результаты, чем раньше. Модель больше не выводит всё в 0.

In [ ]:
print (predictions_train)
print (predictions_test)

In [ ]:
plt.title("Модель со случайной инициализацией")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y[0])

**Наблюдения**:
- Значение функции потерь начинается с большого значения. Причина этого в том, что при больших случайных весах функция активации (sigmoid) выводит результаты, которые очень близки к 0 или 1 для некоторых примеров, и когда он получает этот пример неправильно, значение функции потерь имеет большое значение. Действительно, когда $\log(a^{[3]}) = \log(0)$, потеря идет в бесконечность.
- Плохая инициализация может привести к исчезновению/взрыву градиентов, что также замедляет алгоритм оптимизации.
- Если данную сеть обучать дольше, то вы увидите лучшие результаты, но инициализация с чрезмерно большими случайными числами замедляет оптимизацию.

<font color='blue'>
    
**Что необходимо помнить:**
- Инициализация весов для очень больших случайных значений работает не достаточно хорошо.
- Возможно, что инициализация с небольшими случайными значениями будет лучше. Важный вопрос: насколько малыми должны быть эти случайные величины? 

## 4 - He инициализация

В данной части необходимо реализовать "He инициализацию"; названа в честь первого автора статьи He et al., 2015. (Если вам известно "Xavier инициализация", "He инициализация" похожа за исключением того, что инициализация Xavier использует следующий масштабирующий коэффициент для весов $W^{[l]}$ l слоя `sqrt(1./layers_dims[l-1])`, а He инициализация использует `sqrt(2./layers_dims[l-1])`.)

**Упражнение** Реализовать следующую функцию для "He инициализации".

**Подсказка**: Данная функция похожа на предыдущую `initialize_parameters_random(...)`. Разница лишь в том, что вместо умножения `np.random.randn(..,..)` на 10, необходимо умножить на $\sqrt{\frac{2}{\text{размер предыдущего слоя}}}$, именно это He рекомендует для слоев с функцией активацией ReLU. 

In [ ]:
# ОЦЕНИВАЕМОЕ: initialize_parameters_he

def initialize_parameters_he(layers_dims):
    """
    Arguments:
    layer_dims -- python массив (list), который содержит информацию о размере каждого слоя.
    
    Returns:
    parameters -- python словарь с параметрами "W1", "b1", ..., "WL", "bL":
                    W1 -- матрица весов с размером 1-го слоя (layers_dims[1], layers_dims[0])
                    b1 -- вектор смещений 1-го слоя (layers_dims[1], 1)
                    ...
                    WL -- матрица весов с размером L-го слоя (layers_dims[L], layers_dims[L-1])
                    bL -- вектор смещений L-го слоя (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1
     
    for l in range(1, L + 1):
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (≈ 2 строки кода)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
    return parameters

In [ ]:
parameters = initialize_parameters_he([2, 4, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Ожидаемый результат**:

<table> 
    <tr>
    <td>
    **W1**
    </td>
        <td>
    [[ 1.78862847  0.43650985]
 [ 0.09649747 -1.8634927 ]
 [-0.2773882  -0.35475898]
 [-0.08274148 -0.62700068]]
    </td>
    </tr>
    <tr>
    <td>
    **b1**
    </td>
        <td>
    [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    **W2**
    </td>
        <td>
    [[-0.03098412 -0.33744411 -0.92904268  0.62552248]]
    </td>
    </tr>
    <tr>
    <td>
    **b2**
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>

</table> 

Запустите следующий код для обучения модели на 15 000 итерациях с использованием He инициализацией.

In [ ]:
parameters = model(train_X, train_Y, initialization = "he")
print ("Обучающий датасет:")
predictions_train = predict(train_X, train_Y, parameters)
print ("Тестовый датасет:")
predictions_test = predict(test_X, test_Y, parameters)

In [ ]:
plt.title("Модель He инициализацией")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y[0])

**Наблюдения**:
- Модель с He инициализацией разделяет красные и синие точки достаточно хорошо за небольшое количество итераций.

## 5 - Выводы

Было исследовано три различных типа инициализации. Для одного и того же числа итераций и одних и тех же гиперпараметров:

<table> 
    <tr>
        <td>
        **Модель**
        </td>
        <td>
        **Точность на обучающей выборке**
        </td>
        <td>
        **Проблемма/Комментарий**
        </td>
    </tr>
        <td>
        3-слойнная NN с инициализацией в ноль
        </td>
        <td>
        50%
        </td>
        <td>
        проблема в разрушении симметрии
        </td>
    <tr>
        <td>
        3-слойнная NN со случайной инициализацией
        </td>
        <td>
        83%
        </td>
        <td>
        большие веса при инициализации 
        </td>
    </tr>
    <tr>
        <td>
        33-слойнная NN с He инициализацией
        </td>
        <td>
        99%
        </td>
        <td>
        рекомендуемый метод
        </td>
    </tr>
</table> 

<font color='blue'>
    
**Что необходимо помнить**:
- Разные инициализации приводят к разным результатам
- Случайная инициализация используется, чтобы нарушить симметрию и убедиться, что различные скрытые нейронные могут обучиться разным вещам
- He инициализация хорошо работает для сетей с функцией активации ReLU.

**Ссылки**:
- Курс Deep Learning; https://www.coursera.org/specializations/deep-learning